# Car Rental Data Exploration

This notebook explores the car rental datasets to understand their structure and content before processing them with Spark on EMR.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

## Load the Datasets

Let's load the four datasets: vehicles, users, locations, and rental transactions.

In [ ]:
# Define the data directory
data_dir = '../data'

# Load the datasets
vehicles_df = pd.read_csv(os.path.join(data_dir, 'vehicles.csv'))
users_df = pd.read_csv(os.path.join(data_dir, 'users.csv'))
locations_df = pd.read_csv(os.path.join(data_dir, 'locations.csv'))
transactions_df = pd.read_csv(os.path.join(data_dir, 'rental_transactions.csv'))

print(f"Vehicles dataset shape: {vehicles_df.shape}")
print(f"Users dataset shape: {users_df.shape}")
print(f"Locations dataset shape: {locations_df.shape}")
print(f"Rental transactions dataset shape: {transactions_df.shape}")

## Explore the Vehicles Dataset

In [ ]:
# Display the first few rows of the vehicles dataset
vehicles_df.head()

In [ ]:
# Check for missing values
vehicles_df.isnull().sum()

In [ ]:
# Analyze vehicle types
vehicle_type_counts = vehicles_df['vehicle_type'].value_counts()
print(vehicle_type_counts)

# Plot vehicle types
plt.figure(figsize=(10, 6))
vehicle_type_counts.plot(kind='bar')
plt.title('Vehicle Types Distribution')
plt.xlabel('Vehicle Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze vehicle brands
brand_counts = vehicles_df['brand'].value_counts().head(10)
print(brand_counts)

# Plot top 10 vehicle brands
plt.figure(figsize=(12, 6))
brand_counts.plot(kind='bar')
plt.title('Top 10 Vehicle Brands')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Explore the Users Dataset

In [ ]:
# Display the first few rows of the users dataset
users_df.head()

In [ ]:
# Check for missing values
users_df.isnull().sum()

In [ ]:
# Analyze active users
active_users_count = users_df['is_active'].value_counts()
print(active_users_count)

# Plot active users
plt.figure(figsize=(8, 6))
active_users_count.plot(kind='pie', autopct='%1.1f%%')
plt.title('Active vs. Inactive Users')
plt.ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
# Convert creation_date to datetime
users_df['creation_date'] = pd.to_datetime(users_df['creation_date'])

# Analyze user sign-ups by month
users_df['signup_month'] = users_df['creation_date'].dt.to_period('M')
signup_by_month = users_df.groupby('signup_month').size()

# Plot user sign-ups by month
plt.figure(figsize=(12, 6))
signup_by_month.plot(kind='line', marker='o')
plt.title('User Sign-ups by Month')
plt.xlabel('Month')
plt.ylabel('Number of Sign-ups')
plt.grid(True)
plt.tight_layout()
plt.show()

## Explore the Locations Dataset

In [ ]:
# Display the first few rows of the locations dataset
locations_df.head()

In [ ]:
# Check for missing values
locations_df.isnull().sum()

In [ ]:
# Analyze locations by state
locations_by_state = locations_df['state'].value_counts()
print(locations_by_state)

# Plot locations by state
plt.figure(figsize=(12, 6))
locations_by_state.plot(kind='bar')
plt.title('Locations by State')
plt.xlabel('State')
plt.ylabel('Number of Locations')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze locations by city (top 10)
locations_by_city = locations_df['city'].value_counts().head(10)
print(locations_by_city)

# Plot top 10 cities by number of locations
plt.figure(figsize=(12, 6))
locations_by_city.plot(kind='bar')
plt.title('Top 10 Cities by Number of Locations')
plt.xlabel('City')
plt.ylabel('Number of Locations')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Explore the Rental Transactions Dataset

In [ ]:
# Display the first few rows of the rental transactions dataset
transactions_df.head()

In [ ]:
# Check for missing values
transactions_df.isnull().sum()

In [ ]:
# Convert rental start and end times to datetime
transactions_df['rental_start_time'] = pd.to_datetime(transactions_df['rental_start_time'])
transactions_df['rental_end_time'] = pd.to_datetime(transactions_df['rental_end_time'])

# Calculate rental duration in hours
transactions_df['rental_duration_hours'] = (transactions_df['rental_end_time'] - transactions_df['rental_start_time']).dt.total_seconds() / 3600

# Display summary statistics for rental duration and total amount
print("Rental Duration (hours) Statistics:")
print(transactions_df['rental_duration_hours'].describe())
print("\nTotal Amount Statistics:")
print(transactions_df['total_amount'].describe())

In [ ]:
# Analyze transactions by day of week
transactions_df['day_of_week'] = transactions_df['rental_start_time'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
transactions_by_day = transactions_df['day_of_week'].value_counts().reindex(day_order)

# Plot transactions by day of week
plt.figure(figsize=(12, 6))
transactions_by_day.plot(kind='bar')
plt.title('Rental Transactions by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Transactions')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze transactions by hour of day
transactions_df['hour_of_day'] = transactions_df['rental_start_time'].dt.hour
transactions_by_hour = transactions_df.groupby('hour_of_day').size()

# Plot transactions by hour of day
plt.figure(figsize=(12, 6))
transactions_by_hour.plot(kind='line', marker='o')
plt.title('Rental Transactions by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Transactions')
plt.xticks(range(0, 24))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze distribution of total amount
plt.figure(figsize=(12, 6))
sns.histplot(transactions_df['total_amount'], bins=30, kde=True)
plt.title('Distribution of Rental Transaction Amounts')
plt.xlabel('Total Amount ($)')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze relationship between rental duration and total amount
plt.figure(figsize=(12, 6))
sns.scatterplot(x='rental_duration_hours', y='total_amount', data=transactions_df, alpha=0.5)
plt.title('Relationship Between Rental Duration and Total Amount')
plt.xlabel('Rental Duration (hours)')
plt.ylabel('Total Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

## Cross-Dataset Analysis

In [ ]:
# Merge transactions with vehicles to analyze by vehicle type
transactions_vehicles = pd.merge(transactions_df, vehicles_df[['vehicle_id', 'brand', 'vehicle_type']], on='vehicle_id', how='inner')

# Analyze average transaction amount by vehicle type
avg_amount_by_vehicle_type = transactions_vehicles.groupby('vehicle_type')['total_amount'].mean().sort_values(ascending=False)
print(avg_amount_by_vehicle_type)

# Plot average transaction amount by vehicle type
plt.figure(figsize=(12, 6))
avg_amount_by_vehicle_type.plot(kind='bar')
plt.title('Average Transaction Amount by Vehicle Type')
plt.xlabel('Vehicle Type')
plt.ylabel('Average Amount ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze top 10 pickup locations by transaction count
top_pickup_locations = transactions_df['pickup_location'].value_counts().head(10)
top_pickup_locations_df = pd.DataFrame({'location_id': top_pickup_locations.index, 'transaction_count': top_pickup_locations.values})
top_pickup_locations_with_info = pd.merge(top_pickup_locations_df, locations_df, on='location_id', how='inner')
print(top_pickup_locations_with_info[['location_id', 'location_name', 'city', 'state', 'transaction_count']])

# Plot top 10 pickup locations
plt.figure(figsize=(14, 6))
sns.barplot(x='location_name', y='transaction_count', data=top_pickup_locations_with_info)
plt.title('Top 10 Pickup Locations by Transaction Count')
plt.xlabel('Location Name')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze top 10 users by number of transactions
top_users = transactions_df['user_id'].value_counts().head(10)
top_users_df = pd.DataFrame({'user_id': top_users.index, 'transaction_count': top_users.values})
top_users_with_info = pd.merge(top_users_df, users_df, on='user_id', how='inner')
print(top_users_with_info[['user_id', 'first_name', 'last_name', 'transaction_count']])

# Calculate total spending for top users
top_user_spending = transactions_df[transactions_df['user_id'].isin(top_users.index)].groupby('user_id')['total_amount'].sum().reset_index()
top_users_complete = pd.merge(top_users_with_info, top_user_spending, on='user_id', how='inner')

# Plot top 10 users by total spending
plt.figure(figsize=(14, 6))
sns.barplot(x='user_id', y='total_amount', data=top_users_complete)
plt.title('Top 10 Users by Total Spending')
plt.xlabel('User ID')
plt.ylabel('Total Spending ($)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Summary of Findings

Based on the exploratory data analysis, here are some key findings:

1. Vehicle distribution by type and brand
2. User activity patterns and sign-up trends
3. Geographic distribution of rental locations
4. Temporal patterns in rental transactions (day of week, hour of day)
5. Relationship between rental duration and transaction amount
6. Performance metrics by vehicle type
7. Top performing locations and users

These insights will guide our Spark jobs on EMR to extract meaningful business metrics from the data.